# Semantic Kernel MCP Orchestrator Demo

This notebook demonstrates how to use the Semantic Kernel orchestrator to call the MCP RAG server.

## Prerequisites
- Make sure your MCP server is running on http://127.0.0.1:8002
- Configure your Azure OpenAI or OpenAI credentials in .env
- Install all dependencies using `poetry install`

In [ ]:
import mermaid

graph TD
    A[React Frontend] --> B[FastAPI Server :8000]
    B --> C[ProductChatAgent]
    C --> D[ChatCompletionAgent]
    D --> E[MCPStreamableHttpPlugin]
    E --> F[MCP RAG Server :8002]
    F --> G[ChromaDB Vector Search]
    F --> H[SerpAPI Web Search]
    F --> I[OpenAI GPT-4 for RAG]

SyntaxError: invalid syntax (1893847133.py, line 3)

: 

In [3]:
# Import required libraries
import sys
import os
import asyncio
import json
from pathlib import Path

# Add the project root to Python path
project_root = Path('.').resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[:3]}...")

Project root: C:\Users\aprilhazel\Source\sk_mcp_demo\notebooks
Python path: ['C:\\Users\\aprilhazel\\Source\\sk_mcp_demo\\notebooks', 'C:\\Users\\aprilhazel\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\aprilhazel\\AppData\\Local\\Programs\\Python\\Python312\\DLLs']...


In [ ]:
# Import our modules
from src.utils.config import Config
from src.agents.sk_product_chat_agent import ProductChatAgent

print("✅ Modules imported successfully")

ModuleNotFoundError: No module named 'src.utils.config'

In [ ]:
# Initialize configuration and orchestrator
config = Config()
orchestrator = ProductChatAgent(config)

print("Configuration:")
print(f"  Environment: {config.environment}")
print(f"  MCP Server URL: {config.mcp_server_url}")
print(f"  AI Service: {config.openai_api_type}")
print(f"  Temperature: {config.sk_temperature}")
print("\n✅ Orchestrator initialized!")

## 1. Health Check
First, let's verify that everything is working correctly.

In [ ]:
# Check orchestrator status
status = orchestrator.get_status()
print("📊 Orchestrator Status:")
print(json.dumps(status, indent=2))

In [ ]:
# Check MCP server health
health_result = await orchestrator.mcp_plugin.health_check()
print(f"🏥 MCP Server Health: {health_result}")

## 2. Simple Product Search
Let's search for products in the internal vector database.

In [ ]:
# Search for laptop computers
search_query = "laptop computers"
search_result = await orchestrator.simple_search(search_query, limit=5)

print(f"🔍 Search Results for '{search_query}':")
print(json.dumps(search_result, indent=2))

In [ ]:
# Search for wireless headphones
search_query = "wireless headphones"
search_result = await orchestrator.simple_search(search_query, limit=3)

print(f"🔍 Search Results for '{search_query}':")
if "error" in search_result:
    print(f"❌ Error: {search_result['error']}")
else:
    results = search_result.get('results', [])
    print(f"Found {len(results)} results:")
    for i, item in enumerate(results, 1):
        name = item.get('name', 'Unknown')
        score = item.get('similarity_score', 'N/A')
        print(f"  {i}. {name} (Score: {score})")

## 3. Simple Chat Response
Generate chat responses using the MCP server without full orchestration.

In [ ]:
# Simple chat about laptops
question = "What laptops do you have available for gaming?"
chat_result = await orchestrator.simple_chat(
    question=question,
    use_web_search=True,
    use_evaluation=False
)

print(f"❓ Question: {question}")
print("-" * 50)

if "error" in chat_result:
    print(f"❌ Error: {chat_result['error']}")
else:
    response = chat_result.get('response', 'No response')
    print(f"🤖 Response: {response}")
    
    # Show sources if available
    sources = chat_result.get('sources', {})
    if sources:
        internal_count = len(sources.get('internal', []))
        web_count = len(sources.get('web', []))
        print(f"\n📚 Sources: {internal_count} internal, {web_count} web")

## 4. Evaluated Chat Response
Generate chat responses with evaluation and risk scoring.

In [ ]:
# Evaluated chat with risk scoring
question = "What are the best gaming laptops with RTX 4080 graphics cards?"
eval_result = await orchestrator.simple_chat(
    question=question,
    use_web_search=True,
    use_evaluation=True
)

print(f"❓ Question: {question}")
print("-" * 50)

if "error" in eval_result:
    print(f"❌ Error: {eval_result['error']}")
else:
    response = eval_result.get('response', 'No response')
    print(f"🤖 Response: {response}")
    
    # Show evaluation metrics if available
    evaluation = eval_result.get('evaluation', {})
    if evaluation:
        print("\n📊 Evaluation Metrics:")
        print(f"  • Confidence Score: {evaluation.get('confidence_score', 'N/A')}")
        print(f"  • Risk Score: {evaluation.get('risk_score', 'N/A')}")
        print(f"  • Hallucination Risk: {evaluation.get('hallucination_risk', 'N/A')}")

## 5. Full Orchestrated Response
Use the Semantic Kernel planner for intelligent orchestration of multiple MCP tools.

In [ ]:
# Full orchestrated response
question = "I need a powerful laptop for machine learning work. What do you recommend?"
context = "I'm a data scientist and need something with good GPU support and lots of RAM"

orchestrated_result = await orchestrator.process_question(
    question=question,
    context=context,
    use_evaluation=True
)

print(f"❓ Question: {question}")
print(f"📝 Context: {context}")
print("-" * 50)

if orchestrated_result.get("status") == "failed":
    print(f"❌ Error: {orchestrated_result.get('error', 'Unknown error')}")
    if orchestrated_result.get('details'):
        print(f"Details: {orchestrated_result['details']}")
else:
    response = orchestrated_result.get('response', {})
    if isinstance(response, dict):
        main_response = response.get('response', str(response))
    else:
        main_response = str(response)
    
    print(f"🎼 Orchestrated Response: {main_response}")
    print(f"\n📊 Status: {orchestrated_result.get('status', 'unknown')}")
    print(f"🤖 Orchestrator: {orchestrated_result.get('orchestrator', 'unknown')}")

## 6. Available Functions
Let's see what functions are available in the orchestrator.

In [ ]:
# Get available functions
functions = await orchestrator.get_available_functions()
print("🔧 Available Functions:")
print(json.dumps(functions, indent=2))

## 7. Interactive Testing
Create a simple interactive interface to test different questions.

In [ ]:
# Test different types of questions
test_questions = [
    "What gaming keyboards do you have?",
    "Tell me about the latest AI developments in 2024",
    "What's the best laptop for video editing?",
    "Do you have any wireless mice with good battery life?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"Test Question {i}: {question}")
    print('='*60)
    
    # Use orchestrated response for comprehensive answers
    result = await orchestrator.process_question(
        question=question,
        use_evaluation=True
    )
    
    if result.get("status") == "failed":
        print(f"❌ Error: {result.get('error', 'Unknown error')}")
    else:
        response = result.get('response', {})
        if isinstance(response, dict):
            main_response = response.get('response', str(response))
        else:
            main_response = str(response)
        
        print(f"🤖 Response: {main_response}")
        print(f"Status: {result.get('status', 'unknown')}")

## 8. Performance and Diagnostics
Check the performance and status of the system.

In [ ]:
# Final status check
print("🏁 Final System Status:")
print("="*50)

# Orchestrator status
status = orchestrator.get_status()
print("📊 Orchestrator:")
for key, value in status.items():
    print(f"  • {key}: {value}")

# MCP server health
health = await orchestrator.mcp_plugin.health_check()
print(f"\n🏥 MCP Server: {health}")

print("\n✅ Demo completed successfully!")